# FIRM_DEMO: Capital–Labor Dynamics → Evolutionary Game Analysis (synthetic)

This notebook demonstrates a value‑neutral mapping of firm dynamics (capital–labor allocation)
to an evolutionary game using the `gamify-timeseries` scaffold.

We will:
1) Build **allocation shares** `X` over {LaborComp, CapEx, R&D, SG&A, Payouts}.
2) Define two payoffs: (A) **distributional surplus‑capture**; (B) **value‑gradient to future ROIC**.
3) Learn strategies via `nmf_on_X`, estimate `A` with `estimate_A_from_series`, search ESS via `find_ESS`.
4) Compute a falsifiable statistic for **systematic surplus capture** and test against a row‑shift null.

**Note:** Synthetic data; real analysis requires careful prep, rolling windows, nulls, and intervention checks.

In [ ]:
import sys, os, numpy as np, pandas as pd, matplotlib.pyplot as plt
sys.path.append('/mnt/data')
from gameify_timeseries import (
    nmf_on_X, value_gradient_payoffs, estimate_A_from_series, find_ESS
)
print('Imports OK')

## 1) Simulate a quarterly firm panel

In [ ]:
def simulate_firm(T=80, seed=0):
    rng = np.random.default_rng(seed)
    t = np.arange(T)
    VA = np.exp(0.01*t + 0.05*rng.standard_normal(T)).astype(float)  # value added
    lab_share = 0.58 + 0.05*np.sin(2*np.pi*t/24) + 0.03*rng.standard_normal(T)
    lab_share = np.clip(lab_share, 0.45, 0.70)
    cap_share = 1.0 - lab_share
    W = lab_share * VA                 # labor compensation
    P = cap_share * VA                 # operating surplus
    N = 5
    X = np.zeros((N, T))               # allocation shares
    X[0] = 0.45 + 0.05*np.sin(2*np.pi*(t+3)/20) + 0.02*rng.standard_normal(T)   # LaborComp
    X[1] = 0.20 + 0.03*np.sin(2*np.pi*(t+5)/16) + 0.02*rng.standard_normal(T)   # CapEx
    X[2] = 0.08 + 0.02*np.sin(2*np.pi*(t+8)/18) + 0.01*rng.standard_normal(T)   # R&D
    X[3] = 0.17 + 0.02*np.sin(2*np.pi*(t+2)/26) + 0.02*rng.standard_normal(T)   # SG&A
    X[4] = 0.10 + 0.02*np.sin(2*np.pi*(t+1)/22) + 0.02*rng.standard_normal(T)   # Payouts
    X = np.clip(X, 1e-6, None); X /= X.sum(axis=0, keepdims=True)
    roic = 0.08 + 0.01*np.sin(2*np.pi*t/12) \
           + 0.04*(0.3*X[2] + 0.2*X[1] - 0.1*X[4] - 0.05*X[3]) \
           + 0.01*rng.standard_normal(T)
    return X, VA, W, P, roic

X, VA, W, P, roic = simulate_firm(T=80, seed=2)
players = ['LaborComp','CapEx','R&D','SG&A','Payouts']
print('X shape:', X.shape)

## 2) Payoff A — distributional surplus‑capture proxy
We use Δlog of labor and capital shares of value‑added, assign to rows, and center across players.

In [ ]:
def dlog(arr):
    arr = np.asarray(arr, dtype=float)
    return np.diff(np.log(arr + 1e-12), prepend=np.log(arr[0] + 1e-12))

labor_share = W / (VA + 1e-12)
capital_share = P / (VA + 1e-12)
v_dist = np.zeros_like(X)
v_dist[0] = dlog(labor_share)
v_dist[4] = dlog(capital_share)

N = X.shape[0]
M_I = np.ones((N, N))/N; M_Z = np.eye(N) - M_I
vZ_dist = M_Z @ v_dist
print('v_dist shape:', v_dist.shape)

### Distributional test statistic and null
Statistic: \( D = \overline{v^{Z}_{\text{Payouts}}} - \overline{v^{Z}_{\text{Labor}}} \). Compare to a row‑shift null.

In [ ]:
def circular_shift_rows(M, seed=0):
    rng = np.random.default_rng(seed)
    N, T = M.shape
    Y = np.zeros_like(M)
    for i in range(N):
        k = int(rng.integers(0, T))
        Y[i] = np.roll(M[i], k)
    return Y

D = vZ_dist[4].mean() - vZ_dist[0].mean()
Ds = []
for b in range(500):
    vZ_null = M_Z @ circular_shift_rows(v_dist, seed=b)
    Ds.append(vZ_null[4].mean() - vZ_null[0].mean())
Ds = np.array(Ds)
z = (D - Ds.mean()) / (Ds.std() + 1e-12)
print('D (capital minus labor, mean Z‑payoff):', round(D, 5), '  z vs null:', round(z, 2))

## 3) Strategies and operator (distributional payoff)

In [ ]:
K = 3
S, H = nmf_on_X(X, k=K, iters=300, seed=1, normalize='l2')
est_dist = estimate_A_from_series(S, X, v_dist, k=K, lambda_=1e-2)
A_dist = est_dist['A']
ess_dist = [r for r in find_ESS(A_dist, tol=1e-8, max_support=K) if r['is_ess']]
As, Aa = 0.5*(A_dist + A_dist.T), 0.5*(A_dist - A_dist.T)
print('R^2 (dist):', round(est_dist['R2'], 3), ' ESS:', len(ess_dist), '  skew ratio:', round(np.linalg.norm(Aa)/(np.linalg.norm(A_dist)+1e-12),3))

### Plot inferred strategy mixture `x(t)`

In [ ]:
plt.figure(figsize=(8,3))
for i in range(K):
    plt.plot(est_dist['Xk'][i], label=f'x_{i+1}')
plt.title('Strategy memberships over time'); plt.legend(); plt.show()

## 4) Payoff B — value‑gradient to future ROIC

In [ ]:
H = 4
J_future = np.roll(roic, -H)
v_val = value_gradient_payoffs(X, J_future, ridge=1e-2, standardize=True)
est_val = estimate_A_from_series(S, X, v_val, k=K, lambda_=1e-2)
A_val = est_val['A']
ess_val = [r for r in find_ESS(A_val, tol=1e-8, max_support=K) if r['is_ess']]
Asv, Aav = 0.5*(A_val + A_val.T), 0.5*(A_val - A_val.T)
print('R^2 (value):', round(est_val['R2'], 3), ' ESS:', len(ess_val), '  skew ratio:', round(np.linalg.norm(Aav)/(np.linalg.norm(A_val)+1e-12),3))

## 5) Null test (row shifts) for value‑gradient fit

In [ ]:
def fit_R2_under_null(X, v, S, B=100):
    R2s = []
    for b in range(B):
        v_null = circular_shift_rows(v, seed=100+b)
        est = estimate_A_from_series(S, X, v_null, k=S.shape[1], lambda_=1e-2)
        R2s.append(est['R2'])
    return np.array(R2s)

R2_null = fit_R2_under_null(X, v_val, S, B=100)
z_R2 = (est_val['R2'] - R2_null.mean()) / (R2_null.std() + 1e-12)
print('R^2 z‑score vs row‑shift null:', round(z_R2, 2))